In [20]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import os
from bs4 import BeautifulSoup
#import undetected_chromedriver as uc

In [21]:
# Setup chrome options
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-setuid-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--remote-debugging-port=9222')

# Create service
service = Service('/opt/homebrew/bin/chromedriver')

driver = webdriver.Chrome(service = service, options = options)

In [22]:
# Go to the Glassdoor page while searching for data data science jobs
page_url = "https://www.glassdoor.com/Job/index.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="
driver.get(page_url)

#driver.implicitly_wait(10) # seconds
# Find Job Cards
job_cards = driver.find_elements(By.CLASS_NAME, 'JobCard_trackingLink__zUSOo')

# Get the links list for each card
for card in job_cards:
    card_url = card.get_attribute('href')
    print(card_url)

In [23]:
# Go to each job and get the job details
driver.implicitly_wait(10) # seconds
jobs_list = []
for card in job_cards:
    card_url = card.get_attribute('href')
    driver.get(card_url)

    company = driver.find_element(By.CLASS_NAME, 'EmployerProfile_employerName__8w0tV')
    print(company.text)
    title = driver.find_element(By.CLASS_NAME, 'JobDetails_jobTitle__Rw_gn')
    location = driver.find_element(By.CLASS_NAME, 'JobDetails_location__MbnUM')

    job_details_section = driver.find_element(By.CLASS_NAME, 'JobDetails_jobDescription__6VeBn')
    # Extract the HTML content of the job details section
    html_content = job_details_section.get_attribute("outerHTML")

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Find all paragraphs and list items within the job details section
    description_paragraphs = soup.find_all(["p", "li"])
    
    description = "\n".join(paragraph.get_text() for paragraph in description_paragraphs)
    
    # Add the jobs details to the list
    jobs_list.append({"company": company.text, "location": location.text, "job_title": title.text, "description": description})
    driver.implicitly_wait(2) # seconds
# Close the WebDriver
driver.quit()

In [24]:
# Convert the list to a pandas DataFrame
df = pd.DataFrame(jobs_list)

# Define the file path including the file name
file_path = "Kaggle1.csv"

# Save the DataFrame to an Excel file
df.to_csv(file_path, index=False)

print(f"DataFrame has been saved to {file_path}")

DataFrame has been saved to Kaggle1.csv
